In [1]:
import os
import numpy as np
import pandas as pd

In [2]:
def read_excel(path):
    return pd.read_excel(path)

def delete_outlier(df, HARI, JAM):
    df = df[(df['HARI'] == HARI) & (df['JAM'] == JAM)]    
    
    # Menghitung Q1 (kuartil pertama) dan Q3 (kuartil ketiga)
    Q1 = df['SMP'].quantile(0.25)
    Q3 = df['SMP'].quantile(0.75)
    IQR = Q3 - Q1
    
    # Menentukan batas bawah dan batas atas untuk outlier
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Menemukan dan menghapus outlier
    df_outlier = df[(df['SMP'] < lower_bound) | (df['SMP'] > upper_bound)]
    df_cleaned = df[(df['SMP'] >= lower_bound) & (df['SMP'] <= upper_bound)]
    
    # if(len(df_outlier)>0):
    #     print('Ada Outlier')

    # df_grouped = df_cleaned[['HARI', 'JAM', 'SMP']]
    # df_grouped = round(df_grouped.groupby(['HARI', 'JAM']).mean()).reset_index()

    return df_outlier, df_cleaned


In [3]:
path_1 = '../Data/Preprocessing/Data Losarang/2022/2_input_losarang_2022.xlsx'

df = read_excel(path_1)
df.head()

,WAKTU,TAHUN,BULAN,PEKAN,TANGGAL,HARI,JAM,MOTOR,MOBIL,TRUK/BUS,JUMLAH,SMP_MTR,SMP_MBL,SMP_TRK,SMP
0,2022-01-01 00:00:00,2022,1,1,1,Saturday,00:00,185,39,24,248,148.0,39,60.0,247
1,2022-01-01 00:15:00,2022,1,1,1,Saturday,00:15,195,64,35,294,156.0,64,87.5,308
2,2022-01-01 00:30:00,2022,1,1,1,Saturday,00:30,194,49,17,260,155.2,49,42.5,247
3,2022-01-01 00:45:00,2022,1,1,1,Saturday,00:45,162,43,21,226,129.6,43,52.5,225
4,2022-01-01 01:00:00,2022,1,1,1,Saturday,01:00,125,58,18,201,100.0,58,45.0,203


In [4]:
df_waktu = df[['HARI','JAM']]
df_waktu = df_waktu.groupby(['HARI', 'JAM']).mean().round().reset_index()
df_waktu.head()

,HARI,JAM
0,Friday,00:00
1,Friday,00:15
2,Friday,00:30
3,Friday,00:45
4,Friday,01:00


In [5]:
all_df_outlier = []
all_df_cleaned = []

for i, row in df_waktu.iterrows():
    df_outlier,  df_cleaned = delete_outlier(df, row['HARI'], row['JAM'])
    all_df_outlier.append(df_outlier)
    all_df_cleaned.append(df_cleaned)

# gabungkan data
df_outlier_combined = pd.concat(all_df_outlier)
df_cleaned_combined = pd.concat(all_df_cleaned)

In [6]:
df_outlier_combined.head()

,WAKTU,TAHUN,BULAN,PEKAN,TANGGAL,HARI,JAM,MOTOR,MOBIL,TRUK/BUS,JUMLAH,SMP_MTR,SMP_MBL,SMP_TRK,SMP
18048,2022-07-08 00:00:00,2022,7,2,8,Friday,00:00,153,169,88,410,122.4,169,220.0,511
32160,2022-12-02 00:00:00,2022,12,1,2,Friday,00:00,4,34,23,61,3.2,34,57.5,95
12001,2022-05-06 00:15:00,2022,5,1,6,Friday,00:15,82,225,30,337,65.6,225,75.0,366
3266,2022-02-04 00:30:00,2022,2,1,4,Friday,00:30,14,31,31,76,11.2,31,77.5,120
12002,2022-05-06 00:30:00,2022,5,1,6,Friday,00:30,73,226,29,328,58.4,226,72.5,357


In [7]:
df_cleaned_combined.head()

,WAKTU,TAHUN,BULAN,PEKAN,TANGGAL,HARI,JAM,MOTOR,MOBIL,TRUK/BUS,JUMLAH,SMP_MTR,SMP_MBL,SMP_TRK,SMP
576,2022-01-07,2022,1,1,7,Friday,00:00,35,108,56,199,28.0,108,140.0,276
1248,2022-01-14,2022,1,2,14,Friday,00:00,45,92,61,198,36.0,92,152.5,280
1920,2022-01-21,2022,1,3,21,Friday,00:00,35,100,63,198,28.0,100,157.5,286
2592,2022-01-28,2022,1,4,28,Friday,00:00,28,55,48,131,22.4,55,120.0,197
3264,2022-02-04,2022,2,1,4,Friday,00:00,7,31,30,68,5.6,31,75.0,112


In [8]:
df_outlier_combined = df_outlier_combined.sort_values(by='WAKTU').reset_index(drop=True)
df_cleaned_combined = df_cleaned_combined.sort_values(by='WAKTU').reset_index(drop=True)

In [9]:
df_cleaned_combined.to_excel('../Data/Preprocessing/Data Losarang/2022/3_clean_losarang_2022.xlsx', index=False)
df_outlier_combined.to_excel('../Data/Preprocessing/Data Losarang/2022/3_outliers_losarang_2022.xlsx', index=False)

In [10]:
len(df_cleaned_combined) +  len(df_outlier_combined)

35040

In [11]:
df_cleaned_combined.head()

,WAKTU,TAHUN,BULAN,PEKAN,TANGGAL,HARI,JAM,MOTOR,MOBIL,TRUK/BUS,JUMLAH,SMP_MTR,SMP_MBL,SMP_TRK,SMP
0,2022-01-01 00:00:00,2022,1,1,1,Saturday,00:00,185,39,24,248,148.0,39,60.0,247
1,2022-01-01 00:15:00,2022,1,1,1,Saturday,00:15,195,64,35,294,156.0,64,87.5,308
2,2022-01-01 00:30:00,2022,1,1,1,Saturday,00:30,194,49,17,260,155.2,49,42.5,247
3,2022-01-01 00:45:00,2022,1,1,1,Saturday,00:45,162,43,21,226,129.6,43,52.5,225
4,2022-01-01 01:00:00,2022,1,1,1,Saturday,01:00,125,58,18,201,100.0,58,45.0,203


In [12]:
df_smp = df_cleaned_combined[['WAKTU','TAHUN', 'BULAN', 'TANGGAL', 'HARI','JAM','MOTOR','MOBIL','TRUK/BUS',
                              'SMP_MTR','SMP_MBL','SMP_TRK','SMP']]

In [13]:
df_smp.head(8)

,WAKTU,TAHUN,BULAN,TANGGAL,HARI,JAM,MOTOR,MOBIL,TRUK/BUS,SMP_MTR,SMP_MBL,SMP_TRK,SMP
0,2022-01-01 00:00:00,2022,1,1,Saturday,00:00,185,39,24,148.0,39,60.0,247
1,2022-01-01 00:15:00,2022,1,1,Saturday,00:15,195,64,35,156.0,64,87.5,308
2,2022-01-01 00:30:00,2022,1,1,Saturday,00:30,194,49,17,155.2,49,42.5,247
3,2022-01-01 00:45:00,2022,1,1,Saturday,00:45,162,43,21,129.6,43,52.5,225
4,2022-01-01 01:00:00,2022,1,1,Saturday,01:00,125,58,18,100.0,58,45.0,203
5,2022-01-01 01:15:00,2022,1,1,Saturday,01:15,156,44,17,124.8,44,42.5,211
6,2022-01-01 01:30:00,2022,1,1,Saturday,01:30,150,59,24,120.0,59,60.0,239
7,2022-01-01 01:45:00,2022,1,1,Saturday,01:45,90,40,15,72.0,40,37.5,150


In [14]:
# Round down the WAKTU column to the nearest hour
df_smp_jam = df_smp.copy()
df_smp_jam['JAM'] = df_smp['WAKTU'].dt.floor('H')

# Drop the original WAKTU column if it's not needed anymore
df_smp_jam = df_smp_jam.drop(columns=['WAKTU'])

# Format the 'JAM' column to show only hour and minute
df_smp_jam['JAM'] = df_smp_jam['JAM'].dt.strftime('%H:%M')

# Group by the rounded-down hour and sum the SMP column
df_smp_jam = df_smp_jam.groupby(['TAHUN', 'BULAN', 'TANGGAL', 'HARI', 'JAM'], as_index=False).mean().round()

In [15]:
df_smp_jam.head(8)

,TAHUN,BULAN,TANGGAL,HARI,JAM,MOTOR,MOBIL,TRUK/BUS,SMP_MTR,SMP_MBL,SMP_TRK,SMP
0,2022,1,1,Saturday,00:00,184.0,49.0,24.0,147.0,49.0,61.0,257.0
1,2022,1,1,Saturday,01:00,130.0,50.0,18.0,104.0,50.0,46.0,201.0
2,2022,1,1,Saturday,02:00,89.0,44.0,16.0,71.0,44.0,41.0,157.0
3,2022,1,1,Saturday,03:00,80.0,40.0,13.0,64.0,40.0,33.0,138.0
4,2022,1,1,Saturday,04:00,57.0,31.0,16.0,46.0,31.0,39.0,116.0
5,2022,1,1,Saturday,05:00,74.0,38.0,19.0,59.0,38.0,48.0,145.0
6,2022,1,1,Saturday,06:00,114.0,34.0,36.0,91.0,34.0,89.0,213.0
7,2022,1,1,Saturday,07:00,185.0,56.0,47.0,148.0,56.0,117.0,320.0


In [16]:
# List kolom yang akan dikalikan 4
columns_to_multiply = ['MOTOR', 'MOBIL', 'TRUK/BUS', 'SMP_MTR', 'SMP_MBL', 'SMP_TRK', 'SMP']

# Mengalikan nilai kolom dengan 4
df_smp_jam[columns_to_multiply] = df_smp_jam[columns_to_multiply] * 4

In [17]:
df_lhrt = df_smp_jam.copy()

In [18]:
df_lhrt = df_lhrt.drop(columns=['TAHUN','BULAN','TANGGAL'])

In [19]:
df_lhrt.head()

,HARI,JAM,MOTOR,MOBIL,TRUK/BUS,SMP_MTR,SMP_MBL,SMP_TRK,SMP
0,Saturday,00:00,736.0,196.0,96.0,588.0,196.0,244.0,1028.0
1,Saturday,01:00,520.0,200.0,72.0,416.0,200.0,184.0,804.0
2,Saturday,02:00,356.0,176.0,64.0,284.0,176.0,164.0,628.0
3,Saturday,03:00,320.0,160.0,52.0,256.0,160.0,132.0,552.0
4,Saturday,04:00,228.0,124.0,64.0,184.0,124.0,156.0,464.0


In [20]:
df_lhrt  = df_lhrt.groupby(['HARI', 'JAM'], as_index=False).mean().round()
df_lhrt.head()

,HARI,JAM,MOTOR,MOBIL,TRUK/BUS,SMP_MTR,SMP_MBL,SMP_TRK,SMP
0,Friday,00:00,149.0,245.0,183.0,119.0,245.0,456.0,821.0
1,Friday,01:00,117.0,233.0,164.0,94.0,233.0,408.0,736.0
2,Friday,02:00,106.0,224.0,151.0,84.0,224.0,379.0,688.0
3,Friday,03:00,95.0,208.0,141.0,76.0,208.0,353.0,637.0
4,Friday,04:00,90.0,202.0,134.0,72.0,202.0,334.0,609.0


In [21]:
len(df_lhrt)

168

In [22]:
df_lhrt.to_excel('../Data/Preprocessing/Data Losarang/2022/3_lhrt_losarang_2022.xlsx', index=False)